# Retrieval-Augmented Generation (RAG)

In [1]:
import chromadb
import dotenv
from pathlib import Path
from agents import Agent, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [10]:
# We populated the RAG with the data from the data/calories.csv file in
# the rag_setup.ipynb notebook

chroma_client = chromadb.PersistentClient("../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")
nutrition_qna = chroma_client.get_collection(name="nutrition_qna")

In [3]:
results = nutrition_db.query(query_texts=["banana"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

/home/vscode/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:06<00:00, 13.6MiB/s]


[('calories_per_100g', 89.0), ('food_category', 'fruits'), ('food_item', 'banana'), ('keywords', 'banana_fruits'), ('kj_per_100g', 374.0), ('serving_info', '100g')]
Food: Banana
        Category: Fruits
        Nutritional Information:
        - Calories: 89 per 100g
        - Energy: 374 kJ per 100g
        - Serving size reference: 100g

        This is a fruits food item that provides 89 calories per 100 grams.


[('calories_per_100g', 50.0), ('food_category', '(fruit)juices'), ('food_item', 'banana juice'), ('keywords', 'banana_juice_(fruit)juices'), ('kj_per_100g', 210.0), ('serving_info', '100ml')]
Food: Banana Juice
        Category: (Fruit)Juices
        Nutritional Information:
        - Calories: 50 per 100g
        - Energy: 210 kJ per 100g
        - Serving size reference: 100ml

        This is a (fruit)juices food item that provides 50 calories per 100 grams.




In [32]:
results = nutrition_qna.query(query_texts=["pregnancy"], n_results=2)
for i, doc in enumerate(results["documents"][0]):
    print(sorted(results["metadatas"][0][i].items()))
    print(doc)
    print("\n")

[('is_pregnancy', True)]
Question: What are some possible physical changes that a pregnant woman could experience in the middle months of gestation?
        Answer: During weeks 13-27, you may see an increase in weight and feel more hunger. Backaches might occur frequently, along with leg cramps and heartburn.

        This Q&A pair provides information about nutrition and health topics.


[('is_pregnancy', True)]
Question: What health issues can make a pregnancy more challenging?
        Answer: There are several medical conditions that can potentially increase the risk associated with Pregnancy. These include Anemia during this stage, Hypertensive Disorders related to gestation, Diabetes Mellitus coexisting with Pregnancy, Obesity during pregnancy, as well as Adolescent or Teenage pregnancies.

        This Q&A pair provides information about nutrition and health topics.




In [33]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

In [31]:
calorie_lookup_tool("banana")

'Nutrition Information:\nBanana (Fruits): 89.0 calories per 100g\nBanana Juice ((Fruit)Juices): 50.0 calories per 100g\nBanana (Tropical&Exoticfruits): 89.0 calories per 100g'

In [41]:
@function_tool
def nutrition_qna_tool(query: str, max_results: int = 3):
    """
    Tool function too ask a question about nutrition.

    Args:
        query: The question to ask
        max_results: The maximum number of results to return.

    Returns:
        A string containing the question and the answer related to the query.
    """

    results = nutrition_qna.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):

        formatted_results.append(
            #f"{question} ({answer}): {keywords} calories per 100g"
            doc
        )

    return "QnA Information:\n" + "\n".join(formatted_results)

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `calorie_lookup_tool` function_

In [38]:
nutrition_qna_tool('bananas')

"QnA Information:\nQuestion: Which food is recommended for infants after they've been introduced to ripe bananas and sweet potatoes?\n        Answer: Introduce porridge made from wheat flour or ground rice, starting with only one cereal. Once a week has passed, you may increase the frequency of this new food to two feedings per day.\n\n        This Q&A pair provides information about nutrition and health topics.\nQuestion: What is the recommended amount of bananas I should consume to count it as a single serving?\n        Answer: One small-sized banana can be counted as a single serving.\n\n        This Q&A pair provides information about nutrition and health topics.\nQuestion: Which fruits are high in potassium that I should consider eating?\n        Answer: Avocados, bananas, cantaloups, mangoes and papayas are all excellent options if you're looking for high-potassium fruits to eat.\n\n        This Q&A pair provides information about nutrition and health topics."

In [42]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    If you need to look up calorie information, use the calorie_lookup_tool.
    If are asked a question about nutrition, always use the nutrtition_qna_tool first to see if there is an answer in the knowledge base.
    """,
    tools=[calorie_lookup_tool, nutrition_qna_tool]
)

In [43]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result.final_output)

- Per 100 g: banana 89 kcal, apple 52 kcal.
- Total for a typical medium banana (~118 g) and a medium apple (~182 g): about 200 kcal (banana ~105 kcal + apple ~95 kcal).


In [44]:
with trace("Nutrition Assistant with RAG"):
    result = await Runner.run(
        calorie_agent,
        "How many bananas can a pregenant woman eat in a day?",
    )
    print(result.final_output)

There isn’t a fixed banana limit. Guidance suggests adding about two extra portions per day of fruits and vegetables during pregnancy. You can include bananas as part of those two portions, with variety and overall intake balanced. If you have specific health concerns, consult a clinician.
